# Thermal Compilance shape

### Using the required packages

In [1]:
using GridapTopOpt, Gridap

### Finite Element Parameters

In [2]:
order = 1                               # Finite element order
xmax, ymax = (1.0, 1.0)                 # Domain Size
dom = (0, xmax, 0, ymax)                # Bounding Domain
el_size = (200, 200)                    # Mesh Partition Size
prop_Γ_N = 0.2                          # Γ_N size parameter
prop_Γ_D = 0.2                          # Γ_D size parameter


# Boolean Valued indicator functions

# Γ_N Neumann Boundary Conditions
f_Γ_N(x) = (x[1] ≈ xmax && ymax/2-ymax*prop_Γ_N/2 - eps() <= x[2] <= ymax/2+ymax*prop_Γ_N/2 + eps())
# Γ_D Drichlet Boundary Conditions
f_Γ_D(x) = (x[1] ≈ 0.0 && (x[2] <= ymax*prop_Γ_D + eps() || x[2] >= ymax-ymax*prop_Γ_D/2 - eps()));

### Finite Difference Parameters

In [3]:
# Courant-Friedrichs-Lewy (CFL) number OR
# "Time step coefficient" for Hamilton-Jaccobi equation 
γ = 0.1                                             

 # Reininitialization equation
γ_reinit = 0.5                                     



# "max_steps" and "tol" is scaled by mesh size

# Max steps for advection
max_steps = floor(Int, order*minimum(el_size)/10)  

# Reininitialization tolerance
tol = 1/ (5order^2) /minimum(el_size);             

### Problem Parameters

In [4]:
C = isotropic_elast_tensor(2, 1.0, 0.3)                          # isotropic_elast_tensor(dimension,E,ν)
g = VectorValue(0,-1)                                            # Heat flow in

# κ = 1                                           # Diffusivity
# g = 1                                           # Heat flow in 
vf = 0.4                                        # Volume fraction constraints
lsf_func = initial_lsf(4, 0.2)                  # Initial level set function
iter_mod = 10                                   # VTK Output module
path = joinpath(@__DIR__, "../../../../Result/")
isdir(path) || mkpath(path);

### Model

In [5]:
model = CartesianDiscreteModel(dom, el_size);
update_labels!(1, model, f_Γ_D, "Gamma_D")
update_labels!(2, model, f_Γ_N, "Gamma_N");

In [6]:
# path = joinpath(@__DIR__,"../../../../Result/Before_Triangulation")
# isdir(path) || mkpath(path)
# writevtk(model, joinpath(path, "model_A"));

### Triangulations and Measures

In [7]:
Ω = Triangulation(model)
Γ_N = BoundaryTriangulation(model, tags="Gamma_N")
dΩ = Measure(Ω, 2*order)
dΓ_N = Measure(Γ_N, 2*order);

In [8]:
# path = joinpath(@__DIR__,"../../../../Result/After_Triangulation")
# isdir(path) || mkpath(path)
# writevtk(model, joinpath(path, "model_B"));

### Discretised Finite Element Space

In [9]:
reffe = ReferenceFE(lagrangian,VectorValue{2,Float64},order)
reffe_scalar = ReferenceFE(lagrangian,Float64,order)
V = TestFESpace(model,reffe;dirichlet_tags=["Gamma_D"])
U = TrialFESpace(V,VectorValue(0.0,0.0))
V_φ = TestFESpace(model,reffe_scalar)
V_reg = TestFESpace(model,reffe_scalar;dirichlet_tags=["Gamma_N"])
U_reg = TrialFESpace(V_reg,0);

In [10]:
# path = joinpath(@__DIR__,"../../../../Result/After_Discrete_FE_Space")
# isdir(path) || mkpath(path)
# writevtk(model, joinpath(path, "model_C"));

### Level set and interpolator

In [11]:
φh = interpolate(lsf_func,V_φ)
interp = SmoothErsatzMaterialInterpolation(η = 2*maximum(get_el_Δ(model)))    # η = 2 ×  maximum side length of an element.
I,H,DH,ρ = interp.I,interp.H,interp.DH,interp.ρ;

### Weak formulation

In [12]:
# a(U,V,φ,dΩ,dΓ_N) = ∫((I ∘ φ)*κ*∇(U)·∇(V))dΩ
# l(V,φ,dΩ,dΓ_N) = ∫(g*V)dΓ_N
# state_map = AffineFEStateMap(a, l, U, V, V_φ, U_reg, φh, dΩ, dΓ_N);

In [13]:
a(u,v,φ) = ∫((I ∘ φ)*(C ⊙ ε(u) ⊙ ε(v)))dΩ
l(v,φ) = ∫(v·g)dΓ_N
state_map = AffineFEStateMap(a, l, U, V, V_φ)     #U_reg, φh, dΩ, dΓ_N)

AffineFEStateMap

### Objective and constraints

In [14]:
J(u,φ) = ∫((I ∘ φ)*(C ⊙ ε(u) ⊙ ε(u)))dΩ
dJ(q,u,φ) = ∫((C ⊙ ε(u) ⊙ ε(u))*q*(DH ∘ φ)*(norm ∘ ∇(φ)))dΩ
vol_D = sum(∫(1)dΩ)
C1(u,φ) = ∫(((ρ ∘ φ) - vf)/vol_D)dΩ
dC1(q,u,φ) = ∫(-1/vol_D*q*(DH ∘ φ)*(norm ∘ ∇(φ)))dΩ
pcfs = PDEConstrainedFunctionals(J,[C1],state_map,analytic_dJ=dJ,analytic_dC=[dC1]);

### Velocity extension

In [15]:
α = 4max_steps*γ*maximum(get_el_Δ(model))
a_hilb(p,q) = ∫(α^2*∇(p)⋅∇(q) + p*q)dΩ;
vel_ext = VelocityExtension(a_hilb,U_reg,V_reg)
evo = FiniteDifferenceEvolver(FirstOrderStencil(2,Float64),model,V_φ;max_steps)
reinit = FiniteDifferenceReinitialiser(FirstOrderStencil(2,Float64),model,V_φ;tol,γ_reinit)
ls_evo = LevelSetEvolution(evo,reinit);

###  Optimiser

In [16]:
optimiser = AugmentedLagrangian(pcfs,ls_evo,vel_ext,φh;γ,verbose=true,constraint_names=[:Vol]);

### Solve

In [17]:
path = joinpath(@__DIR__,"../../../../Result/Output/")
isdir(path) || mkpath(path)

true

In [18]:
for (it,uh,φh) in optimiser
    data = ["φ"=>φh,
            "H(φ)"=>(H ∘ φh),
            "|∇(φ)|"=>(norm ∘ ∇(φh)),
            "uh"=>uh]
            
    iszero(it % iter_mod) && writevtk(Ω,path*"_$it",cellfields=data)
    write_history(path*"/history_$max_steps,$tol.txt",get_history(optimiser))
end

Iteration:   0 | L=8.3616e-01, J=8.1524e-01, Vol=2.6359e-01, γ=1.0000e-01, λ1=0.0000e+00, Λ1=6.0242e-01
Iteration:   1 | L=7.7205e-01, J=7.4983e-01, Vol=2.7160e-01, γ=1.0000e-01, λ1=-1.6362e-01, Λ1=6.0242e-01
Iteration:   2 | L=7.6202e-01, J=6.9265e-01, Vol=2.7982e-01, γ=1.0000e-01, λ1=-3.3219e-01, Λ1=6.0242e-01
Iteration:   3 | L=7.6364e-01, J=6.4294e-01, Vol=2.8809e-01, γ=1.0000e-01, λ1=-5.0574e-01, Λ1=6.0242e-01
Iteration:   4 | L=7.7761e-01, J=6.0205e-01, Vol=2.9522e-01, γ=1.0000e-01, λ1=-6.8359e-01, Λ1=6.0242e-01
Iteration:   5 | L=8.0248e-01, J=5.6980e-01, Vol=3.0058e-01, γ=1.0000e-01, λ1=-8.6466e-01, Λ1=6.6266e-01
Iteration:   6 | L=8.3750e-01, J=5.4459e-01, Vol=3.0347e-01, γ=1.0000e-01, λ1=-1.0658e+00, Λ1=6.6266e-01
Iteration:   7 | L=8.7887e-01, J=5.2543e-01, Vol=3.0307e-01, γ=1.0000e-01, λ1=-1.2666e+00, Λ1=6.6266e-01
Iteration:   8 | L=9.1814e-01, J=5.1364e-01, Vol=2.9638e-01, γ=1.0000e-01, λ1=-1.4630e+00, Λ1=6.6266e-01
Iteration:   9 | L=9.4463e-01, J=5.1094e-01, Vol=2.7883e

### Final structure

In [19]:
it = get_history(optimiser).niter; uh = get_state(pcfs)
writevtk(Ω,path*"_Final_$it,$max_steps,$tol", cellfields=["φ"=>φh,"H(φ)"=>(H ∘ φh),"|∇(φ)|"=>(norm ∘ ∇(φh)),"uh"=>uh])

┌ Warning: detected extension '.001' does not correspond to dataset type.
│ Appending '.vtu' to filename.
└ @ WriteVTK C:\Users\IIT BBSR\.julia\packages\WriteVTK\Be3qm\src\WriteVTK.jl:162


(["c:\\Users\\IIT BBSR\\Desktop\\Amiya\\BTP\\Julia\\Level_Set_Method\\Thermal_Compilance\\../../../../Result/Output/_Final_205,20,0.001.vtu"],)